In [ ]:
import pandas as pd
import requests
import os

# Função para buscar o tipo e ID do OSM
def search_address(address):
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": address,
        "format": "json",
        "limit": 1
    }
    headers = {
        "User-Agent": "MyGeocodeApp/1.0 (koretane@gmail.com)"  # Substitua pelo seu email ou informações de contato
    }
    response = requests.get(base_url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data:
            location = data[0]
            return location.get('osm_type'), location.get('osm_id')
        else:
            return None, None
    else:
        return None, None

# Função para obter os detalhes do OSM
def get_details(osm_type, osm_id):
    details_url = f"https://nominatim.openstreetmap.org/details.php?osmtype={osm_type[0].upper()}&osmid={osm_id}&format=json"
    headers = {
        "User-Agent": "MyGeocodeApp/1.0 (koretane@gmail.com)"  # Substitua pelo seu email ou informações de contato
    }
    response = requests.get(details_url, headers=headers)
    if response.status_code == 200:
        details_data = response.json()
        centre_point = details_data.get('centroid', {}).get('coordinates', [None, None])
        return centre_point[1], centre_point[0]  # Invertendo para lat, lon
    else:
        return None, None

# Caminho do arquivo CSV
file_path = r'C:\Users\Public\clientes.csv'

# Ler o arquivo CSV
df = pd.read_csv(file_path, sep=';')

# Lista de endereços
addresses = df['endereco_completo'].tolist()

# Dividir a lista de endereços em 15 partes
chunks = [addresses[i::15] for i in range(15)]

# Processar cada parte e salvar em arquivos Excel separados
for i, chunk in enumerate(chunks):
    output_file_path = os.path.join(r'C:\Users\Public', f'clientes_geocoded_part_{i+1}.xlsx')
    
    # Verificar se o arquivo já existe e pular se existir
    if os.path.exists(output_file_path):
        print(f"Arquivo {output_file_path} já existe. Pulando para a próxima parte.")
        continue
    
    results = []
    for address in chunk:
        osm_type, osm_id = search_address(address)
        if osm_type and osm_id:
            latitude, longitude = get_details(osm_type, osm_id)
            results.append((address, latitude, longitude))
        else:
            results.append((address, None, None))

    # Criar DataFrame com os resultados
    df_results = pd.DataFrame(results, columns=['endereco_completo', 'latitude', 'longitude'])

    # Separar as colunas de endereço completo em logradouro, número, cidade e uf
    df_results[['logradouro', 'numero', 'cidade', 'uf']] = df_results['endereco_completo'].str.split(',', expand=True, n=3)

    # Reordenar as colunas
    df_results = df_results[['logradouro', 'numero', 'cidade', 'uf', 'latitude', 'longitude']]

    # Salvar o DataFrame em um arquivo Excel formatado
    df_results.to_excel(output_file_path, index=False)

print("Arquivos Excel salvos com sucesso.")

Arquivo C:\Users\Public\clientes_geocoded_part_1.xlsx já existe. Pulando para a próxima parte.
Arquivo C:\Users\Public\clientes_geocoded_part_2.xlsx já existe. Pulando para a próxima parte.
Arquivo C:\Users\Public\clientes_geocoded_part_3.xlsx já existe. Pulando para a próxima parte.
Arquivo C:\Users\Public\clientes_geocoded_part_4.xlsx já existe. Pulando para a próxima parte.
